# ML and AWS practical

## Logging in and the console

## Creating EC2 instances

### Background
- Key pairs.
- “Instances” (trade off speed vs time?)
- Spec (GPUs? CPU credits)
- Vendor
- Software? Images.
- Pricing (spot, on-demand, etc)
- Storage types (specialist: HDFS = Hadoop Distributed File System)
- Communication
- Security

### Activity:
- Instantiate, choices.
- Connect to instance

## EMR

- copy roughly from https://aws.amazon.com/blogs/big-data/building-a-recommendation-engine-with-spark-ml-on-amazon-emr-using-zeppelin/
(just spark, hadoop)

